# Week 4
#### model_lstm should be tf_idf model

In [135]:
import torch
import random
import numpy as np

from transformers import AutoTokenizer, BertForMaskedLM

In [136]:
def enforce_reproducibility(seed=42):
    # Sets seed manually for both CPU and CUDA
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    # For atomic operations there is currently 
    # no simple way to enforce determinism, as
    # the order of parallel operations is not known.
    # CUDNN
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    # System based
    random.seed(seed)
    np.random.seed(seed)
enforce_reproducibility()

In [137]:
device = torch.device("cpu")
if torch.cuda.is_available():
    device = torch.device("cuda")
device

device(type='cpu')

In [138]:
"""
Error analysis on BERT
"""

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")
bert = BertForMaskedLM.from_pretrained("bert-base-cased")

def get_probs(sentence, word_idx):
    input_ids = tokenizer.encode(sentence)
    input_ids = torch.tensor([input_ids])
    logits = bert(input_ids)[0][0]
    sorted_probs = logits[word_idx].sort(descending=True)

    return sorted_probs


loading configuration file config.json from cache at /Users/elliamorse/.cache/huggingface/hub/models--bert-base-cased/snapshots/a8d257ba9925ef39f3036bfc338acf5283c512d9/config.json
Model config BertConfig {
  "_name_or_path": "bert-base-cased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.23.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 28996
}

loading file vocab.txt from cache at /Users/elliamorse/.cache/huggingface/hub/models--bert-base-cased/snapshots/a8d257ba9925ef39f3036bfc338acf5283c512d9/vocab.t

NameError: name 'cls' is not defined

In [139]:
sentence = f"I {tokenizer.mask_token} to go to the park."
sorted_probs = get_probs(sentence, 6)
for i in range(10):
    token = tokenizer._convert_id_to_token(sorted_probs.indices[i].numpy().tolist())
    conf = sorted_probs.values[i].detach().numpy().tolist()
    print(f"{token}: {conf}")

the: 25.73165512084961
that: 15.051593780517578
a: 14.412827491760254
to: 12.704647064208984
The: 12.030263900756836
my: 11.993605613708496
some: 11.87283706665039
this: 11.3551025390625
da: 10.254737854003906
their: 10.224621772766113


In [ ]:
import scikitplot as skplt

def predict_intent(input_query, model=model_lstm, i2in=i2in_test, verbose=False):
    sv = query_to_vector(input_query)
     sv = sv.reshape(1,len(sv))
    intent_idx = np.argmax(model.predict(sv), axis=1)[0]
    intent_label = i2in[intent_idx]
     if verbose:
        print(intent_label)
    return intent_label, intent_idx

def evaluate_intent(queries, true_intents, model):
    predicted_intents = []
     for q in queries:
        intent_label, intent_idx = predict_intent(q, model)
         predicted_intents.append(intent_label)
        skplt.metrics.plot_confusion_matrix(true_intents, predicted_intents, figsize=(12,15))
true_intents = [i2in_test[i] for i in intent_data_label_test]
evaluate_intent(query_data_test, true_intents, model_lstm)

In [ ]:
true_intents = ['flight' if i==14 else 'other' for i in intent_data_label_test]
predicted_intents = []
for q in query_data_test:
    intent_label, intent_idx = predict_intent(q, model_lstm2)
    predicted_intents.append('flight' if intent_idx==1 else 'other')
    
skplt.metrics.plot_confusion_matrix(true_intents, predicted_intents, figsize=(12,15))

In [148]:

from sklearn.metrics import classification_report, confusion_matrix
report = classification_report(Y_VAL, y_pred, output_dict=True)
pd.DataFrame(report).transpose()

precision    recall  f1-score     support
0              0.692695  0.555556  0.616592  495.000000
1              0.629005  0.753535  0.685662  495.000000
accuracy       0.654545  0.654545  0.654545    0.654545
macro avg      0.660850  0.654545  0.651127  990.000000
weighted avg   0.660850  0.654545  0.651127  990.000000

In [152]:
from sklearn import metrics
metrics.confusion_matrix(Y_VAL,  y_pred) #(y_actual, y_predicted)

array([[275, 220],
       [122, 373]])